In [166]:
import pandas as pd
import numpy as np

% matplotlib inline
import matplotlib.pyplot as plt

from datetime import datetime

from pandas.tseries.offsets import *

AFRs based on [Backblaze Q3 2017 HDD Reliability Report](https://www.backblaze.com/blog/hard-drive-failure-rates-q3-2017/) and [2016 Cumulative Reliability Reports](https://www.backblaze.com/blog/hard-drive-benchmark-stats-2016/).

2TB Drives Listed in the 2016 report:

 * Western Digital WC20EFRX; AFR 4.94%
 * Seagate ST320LT007; AFR 47.70%
 * Seagate ST32000542AS; AFR 9.79%
 * HGST HDS722020ALA;  AFR 1.58%
 
3TB Drives:

 * Western Digital WD30EZRX; AFR 7.38%
 * Western Digital WD30EFRX; AFR 5.74%
 * Seagate ST33000651AS; AFR 4.27%
 * Seagate ST3000DM001; AFR 26.72%
 * HGST HDS723030ALA; AFR 1.92%
 * HGST HDS5C3030ALA;  AFR 0.82%
 * Toshba DT01ACA300; AFR 3.96%
 
8TB:

 * HGST HUH728080ALE600; AFR 1.70%
 * Seagate ST8000DM002; AFR 1.10%
 * Seagate ST8000NM0055; AFR 1.2%
 
The 12TB drives are too new to have reliablity stats (Backblaze has 20 instances with 500 hours total), but have a quoted reliablity of 0.35%.  Taken with a grain of salt, we've doubled that


In [256]:

labels = ['model','tb','cost','warranty']
hdds = [('WC20EFRX', 2, 85, 3 ), 
        ('WD30EFRX', 3, 106, 3),
        ('ST12000NM007', 12, 460, 5 )] 
hdd_data = pd.DataFrame.from_records(hdds, columns=labels)

epoch = 2000

afr_horizon = 5 #years
#afr_index = pd.period_range(pd.Period(year=1,freq='A'), pd.Period(year=afr_horizon,freq='A'), freq='A')
afr_index = pd.date_range(datetime(epoch,1,1), datetime(epoch+afr_horizon,1,1), freq='A')
afr_data = pd.DataFrame( index=afr_index )

afr_data['WC20EFRX'] = (0.0494, 0.0494, 0.0494, 0.0494, 0.0494)
afr_data['WD30EFRX'] = (0.0574, 0.0574, 0.0574, 0.0574, 0.0574)
afr_data['ST12000NM007'] = (0.0120, 0.0120, 0.0120, 0.0120, 0.0120)

print(hdd_data)
print(afr_data)

# hdd_data = pd.DataFrame( {'afr': [0.0494, 0.0574, 0.012, 0.007],
#                           'bytes': [2*10**12, 3*10**12, 8*10**12, 12*10**12],
#                            'cost': [85,106,250,460]},
#                            index = ['2tb','3tb','8tb','12tb'])

          model  tb  cost  warranty
0      WC20EFRX   2    85         3
1      WD30EFRX   3   106         3
2  ST12000NM007  12   460         5
            WC20EFRX  WD30EFRX  ST12000NM007
2000-12-31    0.0494    0.0574         0.012
2001-12-31    0.0494    0.0574         0.012
2002-12-31    0.0494    0.0574         0.012
2003-12-31    0.0494    0.0574         0.012
2004-12-31    0.0494    0.0574         0.012


In [257]:
labels=['name','drive','qty','redun','purchased']
arrays = [('covis-nas1', 'WC20EFRX', 4, 1, 2011)]

array_data = pd.DataFrame.from_records( arrays, columns=labels)

print(array_data)

# array_data = pd.DataFrame( columns = ['drive', 'num', 'redun','qty','to_purchase'],
#                           data = [['2tb',4,1,2,0],
#                                   ['3tb',4,1,4,0],
#                                   ['8tb',8,2,1,8 ],
#                                   ['12tb',8,2,1,8] ] )

# ## Look up drive data
# array_data['afr'] = [hdd_data.loc[d].afr for d in array_data['drive']]
# array_data['hdd_cost'] = [hdd_data.loc[d].cost for d in array_data['drive']]
# array_data['bytes'] = [hdd_data.loc[d].bytes for d in array_data['drive']]

         name     drive  qty  redun  purchased
0  covis-nas1  WC20EFRX    4      1       2011


In [295]:

rng = pd.date_range(datetime(2018,1,1), periods=10, freq='A')    
p_survival = pd.DataFrame(index=rng)
                         
#print(p_survival)

for index,array in array_data.iterrows():
    print("Array = %s" % array['name'])
        
    this_afr = afr_data[ array['drive'] ].copy()   
    this_afr.index = this_afr.index + DateOffset(years=array['purchased']-2000)

    ## Extend to match rng
    idx = pd.date_range(this_afr.index.min(),p_survival.index.max(),freq='A')
    print(idx)
    
    # Forward fill to get years we don't have
    this_afr=this_afr.reindex(idx).ffill()

    
    ## Extract the AFRs for the relevant dates
    this_afr=this_afr[p_survival.index]
    this_survival = 1-this_afr
    print(this_survival)
    
    
    drive_p_survival = np.cumprod(this_survival)
    print(drive_p_survival)
    
    p_survival = drive_p_survival ** array['qty']
    
    #this_drive = afr_lookup( array['drive'], age )

Array = covis-nas1
DatetimeIndex(['2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31',
               '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31',
               '2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31',
               '2023-12-31', '2024-12-31', '2025-12-31', '2026-12-31',
               '2027-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')
2018-12-31    0.9506
2019-12-31    0.9506
2020-12-31    0.9506
2021-12-31    0.9506
2022-12-31    0.9506
2023-12-31    0.9506
2024-12-31    0.9506
2025-12-31    0.9506
2026-12-31    0.9506
2027-12-31    0.9506
Freq: A-DEC, Name: WC20EFRX, dtype: float64
2018-12-31    0.950600
2019-12-31    0.903640
2020-12-31    0.859001
2021-12-31    0.816566
2022-12-31    0.776228
2023-12-31    0.737882
2024-12-31    0.701431
2025-12-31    0.666780
2026-12-31    0.633841
2027-12-31    0.602529
Freq: A-DEC, Name: WC20EFRX, dtype: float64


In [86]:
array_data['p_no_drives_fail_1yr'] = (1-array_data.afr)**array_data.num
array_data['p_no_arrays_fail_1yr'] = array_data.p_no_drives_fail_1yr ** array_data.qty

duration = 5

array_data['p_drive_failure_5yr'] = 1-(1-array_data.afr)**duration
array_data['expected_cost_5yr'] = array_data.p_drive_failure_5yr * array_data.num * array_data.qty * array_data.hdd_cost

array_data['p_no_drives_fail_5yr'] = array_data.p_no_drives_fail_1yr ** duration
array_data['p_no_arrays_fail_5yr'] = array_data.p_no_arrays_fail_1yr ** duration

array_data['startup_cost'] = array_data.hdd_cost * array_data.to_purchase
array_data['tco'] = array_data.startup_cost + array_data.expected_cost_5yr

array_data['capacity'] = (array_data.num-array_data.redun) * array_data.bytes * array_data.qty

In [87]:
print(array_data)

  drive  num  redun  qty  to_purchase     afr  hdd_cost         bytes  \
0   2tb    4      1    2            0  0.0494      85.0  2.000000e+12   
1   3tb    4      1    4            0  0.0574     106.0  3.000000e+12   
2   8tb    8      2    1            8  0.0120     250.0  8.000000e+12   
3  12tb    8      2    1            8  0.0070     460.0  1.200000e+13   

   p_no_drives_fail_1yr  p_no_arrays_fail_1yr  p_drive_failure_5yr  \
0              0.816566              0.666780             0.223772   
1              0.789423              0.388364             0.255890   
2              0.907937              0.907937             0.058577   
3              0.945353              0.945353             0.034513   

   expected_cost_5yr  p_no_drives_fail_5yr  p_no_arrays_fail_5yr  \
0         152.165270              0.363041              0.131799   
1         433.989335              0.306583              0.008835   
2         117.154353              0.616990              0.616990   
3         1